In [73]:
import models
from permutation_metrics import rank_similarities
import utils
import torch
import torch.nn as nn
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
from torch import optim
from torch.autograd import Variable
import sklearn.metrics


%load_ext autoreload
%autoreload 2

torch.manual_seed(42)

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

Y = torch.tensor(np.load('../datasets/sigmoid_irf_uncorrel.npy'), dtype=dtype)
Yt = Y.transpose(0,1)
real_data = torch.tensor(np.genfromtxt('../datasets/real_data.csv', delimiter = ','), dtype = dtype)
A_true = np.load('../datasets/students_uncorrel.npy')
D_true = np.load('../datasets/questions_uncorrel.npy')

# We assume we know the relevant concept of each question beforehand
concepts = np.nonzero(D_true)
num_students, num_concepts = A_true.shape
num_questions = D_true.shape[0]
guess_prob = 1/5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
def accuracy(R_pred, R_true):
    R_pred = R_pred.data.numpy()
    R_true = R_true.data.numpy()
    R_pred_cpy = np.copy(R_pred)
    R_pred_cpy[R_pred_cpy > 0.5] = 1
    R_pred_cpy[R_pred_cpy <= 0.5] = 0
    print("Accuracy: {}".format(np.sum(R_pred_cpy == R_true) / (R_true.shape[0]*R_true.shape[1])))
    return np.sum(R_pred_cpy == R_true) / (R_true.shape[0]*R_true.shape[1])

In [19]:
def accuracy_last(R_pred, R_true):
    R_pred = R_pred.data.numpy()
    R_true = R_true.data.numpy()
    R_pred_cpy = np.copy(R_pred)
    R_pred_cpy[R_pred_cpy > 0.5] = 1
    R_pred_cpy[R_pred_cpy <= 0.5] = 0
    print("Accuracy: {}".format(np.sum(R_pred_cpy == R_true) / (R_true.shape[0])))
    return np.sum(R_pred_cpy == R_true) / (R_true.shape[0])

In [20]:
def rmse_min_max(A,B):
    a_norm = (A - A.min())/(A.max()-A.min())
    b_norm = (B - B.min())/(B.max()-B.min())
    return np.sqrt(np.mean(np.square(a_norm-b_norm))) 

# Run on Simulated Data

In [76]:
#Preparing to train simulated data set
observations = (real_data.transpose(0,1))
#observations = Yt
idx_train = int(0.7*observations.size()[1])
idx_val = int(0.8*observations.size()[1])
train = observations[:,:idx_train]
val = observations[:,idx_train:idx_val]
test = observations[:,idx_val:]
idx_test = observations.size()[1] - idx_val
idx_val = idx_val-idx_train


## Simple RNN to predict students performance

In [201]:
n_epochs = 1000
hidden_size = 128
layers = 4
batch_size = 100 #needs to have idx_train, idx_val, and idx_train as a multiple
rate = 0.002
dropout = 0.5

model = models.RNN_Model(hidden_size, batch_size, layers, dropout)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [83]:
#Training
losses = np.zeros(n_epochs) # For plotting
num_batches = int(idx_train/batch_size)
preds = torch.zeros(train.size()[0]-1,train.size()[1])

for epoch in range(n_epochs):
    for batch in range(num_batches):
        inputs = Variable(train[:-1,batch*batch_size:(batch+1)*batch_size])
        targets = Variable(train[1:,batch*batch_size:(batch+1)*batch_size])

        outputs, hidden = model(inputs, None)
        preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
        
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        #if batch % 2 == 0:
            #print("Batch Done")
    if epoch % 20 == 0:
        losses[epoch] += loss.data[0]
        print(epoch, loss.data[0])
        accuracy(preds, train[1:,:])

/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(0.6935)
Accuracy: 0.5069196428571429
20 tensor(0.5866)
Accuracy: 0.70625
40 tensor(0.4788)
Accuracy: 0.74375
60 tensor(0.5297)
Accuracy: 0.7265625
80 tensor(0.4069)
Accuracy: 0.7816964285714286
100 tensor(0.3784)
Accuracy: 0.7875
120 tensor(0.3377)
Accuracy: 0.8296875


KeyboardInterrupt: 

In [84]:
#Validation
num_batches = int(idx_val/batch_size)
preds = torch.zeros(val.size()[0]-1,val.size()[1])

for batch in range(num_batches):
    inputs = Variable(val[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(val[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
    
accuracy(preds, val[1:,:])

Accuracy: 0.6125


0.6125

In [ ]:
#Test
num_batches = int(idx_test/batch_size)
preds = torch.zeros(test.size()[0]-1,test.size()[1])

for batch in range(num_batches):
    inputs = Variable(test[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(test[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs

accuracy(preds, test[1:,:])

## Skill RNN to predict students performance

In [102]:
n_epochs = 500
batch_size = 80
average = True
sigmoid = True
concepts = ([0,1,2,3,4,5,6,7,8],[0,0,0,0,0,0,0,0,0]) #use for real data
num_questions = observations.size()[0] #use for real data
hidden_size = 128
dropout = 0.3
num_layers = 4
rate = 0.002

model = models.RNN_Skills_Model(average, concepts, num_concepts, num_questions, hidden_size, batch_size, num_layers, dropout, sigmoid)
criterion = nn.functional.binary_cross_entropy
optimizer = optim.Adam(model.parameters(), lr=rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [103]:
#Training
losses = np.zeros(n_epochs) # For plotting
num_batches = int(idx_train/batch_size)
preds = torch.zeros(train.size()[0]-1,train.size()[1])
A_pred = torch.zeros(train.size()[1],num_concepts)

for epoch in range(n_epochs):

    num_batches = int(idx_train/batch_size)
    preds = torch.zeros(train.size()[0]-1,train.size()[1])
    A_pred = torch.zeros(train.size()[1],num_concepts)
    for batch in range(num_batches):
        inputs = Variable(train[:-1,batch*batch_size:(batch+1)*batch_size])
        targets = Variable(train[1:,batch*batch_size:(batch+1)*batch_size])

        outputs, hidden, skills, D = model(inputs, None)
        preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
        A_pred[batch*batch_size:(batch+1)*batch_size,:] = skills 
        
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    losses[epoch] += loss.data[0]
    
    

    if epoch % 20 == 0:
        print(epoch, loss.data[0])
        accuracy(preds,train[1:,:])
        #print('RMSE A: {}'.format(rmse_min_max(A_pred.data.numpy(), A_true[:idx_train])))
        #print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))
        
        print("Validation")
        num_batches = int(idx_val/batch_size)
        preds = torch.zeros(val.size()[0]-1,val.size()[1])
        A_pred_val = torch.zeros(val.size()[1],num_concepts)

        for batch in range(num_batches):
            inputs = Variable(val[:-1,batch*batch_size:(batch+1)*batch_size])
            targets = Variable(val[1:,batch*batch_size:(batch+1)*batch_size])

            outputs, hidden, skills, D = model(inputs, None)
            preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
            A_pred_val[batch*batch_size:(batch+1)*batch_size,:] = skills 

        accuracy_last(preds[-1,:], val[-1,:])
        accuracy(preds, val[1:,:])
        #print('RMSE A: {}'.format(rmse_min_max(A_pred_val.data.numpy(), A_true[idx_train:(idx_train+idx_val)])))
        #print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))
        
accuracy(preds,train[1:,:])

/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(0.6815)
Accuracy: 0.5183035714285714
Validation
Accuracy: 0.7625
Accuracy: 0.5875
20 tensor(0.6362)
Accuracy: 0.5183035714285714
Validation
Accuracy: 0.7625
Accuracy: 0.5875
40 tensor(0.6084)
Accuracy: 0.5183035714285714
Validation
Accuracy: 0.7625
Accuracy: 0.5875
60 tensor(0.5858)
Accuracy: 0.5428571428571428
Validation
Accuracy: 0.7625
Accuracy: 0.6296875
80 tensor(0.5675)
Accuracy: 0.6743303571428572
Validation
Accuracy: 0.7625
Accuracy: 0.6875
100 tensor(0.5515)
Accuracy: 0.6979910714285714
Validation
Accuracy: 0.7625
Accuracy: 0.69375
120 tensor(0.5357)
Accuracy: 0.7296875
Validation
Accuracy: 0.7625
Accuracy: 0.690625
140 tensor(0.5228)
Accuracy: 0.7589285714285714
Validation
Accuracy: 0.7625
Accuracy: 0.6875
160 tensor(0.5138)
Accuracy: 0.7649553571428571
Validation
Accuracy: 0.7625
Accuracy: 0.6921875
180 tensor(0.5079)
Accuracy: 0.7680803571428572
Validation
Accuracy: 0.7625
Accuracy: 0.6984375
200 tensor(0.5067)
Accuracy: 0.7767857142857143
Validation
Accuracy: 0.76

0.8274553571428571

In [104]:
#print(len(np.unique(train[1:,:].detach().numpy())))
#print(train)
sklearn.metrics.roc_auc_score(train[1:,:].detach().numpy(),preds.detach().numpy())

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [95]:
results = rank_similarities(A_true[:idx_train],train.transpose(0,1).data.numpy(), A_pred.data.numpy())
print(results['summary'])


    Summary of Ranking Evaluation: 
    Correlations with true rankings derived from A_true.
        Baseline: 
            Kendall:-0.015 (p-value 0.624)
            Spearman: -0.021 (p-value 0.616) 
            
        Prediction:
            Kendall: -0.011 (p-value 0.702)
            Spearman: -0.015 (p-value 0.725)  
            
    Average difference: 0.005 (absolute diff., vs. the baseline) 
    


In [105]:
#Validation
num_batches = int(idx_val/batch_size)
preds = torch.zeros(val.size()[0]-1,val.size()[1])
A_pred = torch.zeros(val.size()[1],num_concepts)

for batch in range(num_batches):
    inputs = Variable(val[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(val[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden, skills, D = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
    A_pred[batch*batch_size:(batch+1)*batch_size,:] = skills 
    
#print('RMSE A: {}'.format(rmse_min_max(A_pred.data.numpy(), A_true[idx_train:(idx_train+idx_val)])))
#print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))
accuracy(preds, val[1:,:])
accuracy_last(preds[-1,:], val[-1,:])

Accuracy: 0.6828125
Accuracy: 0.7625


0.7625

In [106]:
#Test
num_batches = int(idx_test/batch_size)
preds = torch.zeros(test.size()[0]-1,test.size()[1])
A_pred = torch.zeros(test.size()[1],num_concepts)

for batch in range(num_batches):
    inputs = Variable(test[:-1,batch*batch_size:(batch+1)*batch_size])
    targets = Variable(test[1:,batch*batch_size:(batch+1)*batch_size])

    outputs, hidden, skills, D = model(inputs, None)
    preds[:,batch*batch_size:(batch+1)*batch_size] = outputs
    A_pred[batch*batch_size:(batch+1)*batch_size,:] = skills 
    
#print('RMSE A: {}'.format(rmse_min_max(A_pred.data.numpy(), A_true[(idx_train+idx_val):])))
#print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))
accuracy(preds, test[1:,:])
accuracy_last(preds[-1,:], test[-1,:])

Accuracy: 0.6984375
Accuracy: 0.76875


0.76875

In [67]:
results = rank_similarities(A_true[(idx_train+idx_val):], test.transpose(0,1).data.numpy(), A_pred.data.numpy())
print(results['summary'])


    Summary of Ranking Evaluation: 
    Correlations with true rankings derived from A_true.
        Baseline: 
            Kendall:0.706 (p-value 0.0)
            Spearman: 0.88 (p-value 0.0) 
            
        Prediction:
            Kendall: 0.02 (p-value 0.671)
            Spearman: 0.029 (p-value 0.68)  
            
    Average difference: -0.768 (absolute diff., vs. the baseline) 
    


In [14]:
results = rank_similarities(A_true, Yt, train)
print(results['summary'])


        Summary of Ranking Evaluation: 
        The correlations are with the true rankings derived from A_true.
        For the baseline, we get a Kendall Rank correlation of 0.033, with p-value of 0.124, and a Spearman correlation of 0.049, with p-value of 0.125. 
        For the prediction, we get a Kendall Rank correlation of -0.012, with p-value of 0.558, and a Spearman correlation of -0.019, with p-value of 0.546.  
        Which gives us an average difference of -0.0565 versus the baseline. 
        


In [32]:
#Hyperparameter Search
n_epochs = 50
hidden_size = {512,256,128,64}
dropout = np.random.uniform(size=5)
print(dropout)
num_layers = {32,16,8,4}
#l_rate = np.random.uniform(0.01,0.5,5)
l_rate = {0.002,0.01,0.05,0.2,0.5}
print(l_rate)
A_rmse = 1
D_rmse = 1
max_acc = 0
params = {}
concepts = np.nonzero(D_true)
acc = []
h_loss = {}
i=0
for size in hidden_size:
    for drop in dropout:
        for layers in num_layers:
            for rate in l_rate:
                model = models.RNN_Skills_Model(False, concepts, num_concepts, num_questions, size, num_students, layers, drop)
                criterion = nn.functional.binary_cross_entropy
                optimizer = optim.Adam(model.parameters(), lr=rate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

                losses = np.zeros(n_epochs)
                
                for epoch in range(n_epochs):

                    inputs = Variable(Yt[:-1])
                    targets = Variable(Yt[1:])

                    outputs, hidden, skills, D = model(inputs, None)

                    optimizer.zero_grad()
                    loss = criterion(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    
                    losses[epoch] += loss.data[0]
                    
                    if epoch>0 and np.abs(losses[epoch]-losses[epoch-1])<0.00005:
                        break

                    if epoch % 10 == 0:
                        print(epoch, loss.data[0])
                        print(accuracy(outputs,targets))
                        print('RMSE A: {}'.format(rmse_min_max(skills.data.numpy(), A_true)))
                        print('RMSE D: {}'.format(rmse_min_max(D.data.numpy(), D_true)))

                
                print(accuracy(outputs,targets))
                if accuracy(outputs,targets)>max_acc:
                    params['accuracy'] = (size,drop,layers,rate)
                    max_acc = accuracy(outputs,targets)
                if rmse_min_max(skills.data.numpy(), A_true)<A_rmse:
                    params['A'] = (size,drop,layers,rate)
                    A_rmse = rmse_min_max(skills.data.numpy(), A_true)
                if rmse_min_max(D.data.numpy(), D_true)<D_rmse:
                    params['D'] = (size,drop,layers,rate)
                    D_rmse = rmse_min_max(D.data.numpy(), D_true)
                acc.append(accuracy(outputs,targets))
                h_loss[i] = losses
                print('done')
                i+=1
                    
print(params)
print(max_acc)
print(A_rmse)
print(D_rmse)
print(acc)

[0.67363352 0.34875009 0.45567222 0.76393018 0.62340218]
{0.5, 0.1, 0.3, 0.2, 0.02}


/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/juasarmiento10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(1.3280)
0.48944444444444446
RMSE A: 0.41321895522657626
RMSE D: 0.46254148619455304
10 tensor(0.8601)
0.4803131313131313
RMSE A: 0.36741722820092293
RMSE D: 0.3854353724191575
0.4787070707070707
done
0 tensor(1.9985)
0.5032929292929293
RMSE A: 0.50919118529673
RMSE D: 0.4492963114053607
10 tensor(0.8606)
0.4785656565656566
RMSE A: 0.36756588297951304
RMSE D: 0.3713725979540209
20 tensor(0.8554)
0.48397979797979795
RMSE A: 0.3641292753668714
RMSE D: 0.42039191691995553
0.48597979797979796
done
0 tensor(1.2593)
0.5110707070707071
RMSE A: 0.48182128081911146
RMSE D: 0.330943770978714
0.4784242424242424
done
0 tensor(2.4409)
0.5184444444444445
RMSE A: 0.47526985063165056
RMSE D: 0.34508595146288723
10 tensor(0.8406)
0.5003636363636363
RMSE A: 0.35819794728253196
RMSE D: 0.43959553735739854
20 tensor(0.8348)
0.5008181818181818
RMSE A: 0.3194916266610254
RMSE D: 0.4446673889964636
0.5054848484848485
done
0 tensor(1.4675)
0.4865959595959596
RMSE A: 0.4976868465537525
RMSE D: 0.387302

KeyboardInterrupt: 